In [75]:
from helper_code import *
from team_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import os
import tqdm
import numpy as np
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
# Find data files.

data_folder = "./data/dataset/data/training_data/"

# Find the patient data files.
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

if num_patient_files==0:
    raise Exception('No data was provided.')

# Create a folder for the model if it does not already exist.
#os.makedirs(model_folder, exist_ok=True)
#TODO: remove this:
#classes = ['Present', 'Unknown', 'Absent']
#num_classes = len(classes)

murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)
outcome_classes = ['Abnormal', 'Normal']
num_outcome_classes = len(outcome_classes)

# Extract the features and labels.
#if verbose >= 1:
#    print('Extracting features and labels from the Challenge data...')

data = []
murmurs = list()
outcomes = list()
NEW_FREQUENCY = 100

for i in tqdm.tqdm(range(num_patient_files)):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings, freq = load_recordings(data_folder, current_patient_data, get_frequencies=True)
    for j in range(len(current_recordings)):
        data.append(signal.resample(current_recordings[j], int((len(current_recordings[j])/freq[j]) * NEW_FREQUENCY)))
        current_auscultation_location = current_patient_data.split('\n')[1:len(current_recordings)+1][j].split(" ")[0]
        all_murmur_locations = get_murmur_locations(current_patient_data).split("+")
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        if get_murmur(current_patient_data) == "Present":
            if current_auscultation_location in all_murmur_locations:
                current_murmur[0] = 1
            else:
                pass
        elif get_murmur(current_patient_data) == "Unknown":
            current_murmur[1] = 1
        elif get_murmur(current_patient_data) == "Absent":
            current_murmur[2] = 1
        murmurs.append(current_murmur)

        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)

data_padded = pad_array(data)



100%|██████████| 942/942 [00:31<00:00, 29.70it/s]


In [74]:
np.expand_dims(data_padded,2).shape

(3163, 6451, 1)

In [70]:
data_padded.shape

(3163, 6451)

In [65]:
def pad_array(data):
    max_len = 0
    for i in data:
        if len(i) > max_len:
            max_len = len(i)
    new_arr = np.zeros((len(data),max_len))
    for j in range(len(data)):
        new_arr[j,:len(data[j])] = data[j]
    return new_arr

In [ ]:
max_len = 0
for i in data:
    if len(i) > max_len:
        max_len = len(i)

In [53]:
len(data)

3163

In [55]:
new_arr = np.zeros((len(data),max_len))

In [56]:
for j in range(len(data)):
    new_arr[j,:len(data[j])] = data[j]
    

In [59]:
import matplotlib.pyplot as plt

In [52]:
keras.utils.pad_sequences(data, padding='pre', truncating='pre', value=0.0, maxlen=32256)

NameError: name 'keras' is not defined

In [47]:
max_len = 0
for i in data:
    if len(i) > max_len:
        max_len = len(i)


In [48]:
max_len

32256

In [46]:
len(murmurs)

3163

In [42]:
np.unique(np.argmax(murmurs, axis=1), return_counts=True)

(array([0, 1, 2], dtype=int64), array([ 616,  156, 2391], dtype=int64))

In [76]:
np.unique(np.argmax(outcomes, axis=1), return_counts=True)

(array([0, 1], dtype=int64), array([1531, 1632], dtype=int64))

In [35]:
get_murmur_locations(current_patient_data).split("+")

['nan']

In [79]:
np.argmax(np.array([[1,0,0],[0,1,0],[0,0,1]]),axis=1)

array([0, 1, 2], dtype=int64)

In [87]:
np.sum(np.array([[0.2,0.7,0.1],[0.3,0.5,0.2],[0.1,0.8,0.1]]),axis=0)/3

array([0.2       , 0.66666667, 0.13333333])

In [82]:
np.sum(np.array([0.4,0.3,0.5,0.7,0.9]))/5

0.5599999999999999

In [ ]:
np.array([[1,0,0],[0,1,0],[0,0,1]])

In [19]:
current_patient_data.split('\n')[1:4+1][0].split(" ")[0]

'AV'

In [16]:
current_recordings

[array([1653, 3815, 3101, ...,  187,  348,   53], dtype=int16),
 array([32596, 28290,  8851, ...,  6449,  6299,  7167], dtype=int16),
 array([   244,  -3588, -22579, ...,   -207,   -205,   -204], dtype=int16),
 array([5039, 5952, 5754, ...,  344,  229,   87], dtype=int16)]

In [15]:
load_patient_data(patient_files[i])

'9979 4 4000\nAV 9979_AV.hea 9979_AV.wav 9979_AV.tsv\nPV 9979_PV.hea 9979_PV.wav 9979_PV.tsv\nTV 9979_TV.hea 9979_TV.wav 9979_TV.tsv\nMV 9979_MV.hea 9979_MV.wav 9979_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 103.0\n#Weight: 13.1\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: AV+MV+PV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Diamond\n#Systolic murmur grading: III/VI\n#Systolic murmur pitch: High\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [23]:
current_location = current_patient_data.split('\n')[1:len(current_recordings)+1][j].split(" ")[0]

In [24]:
all_murmur_locations = get_murmur_locations(current_patient_data).split("+")

In [28]:
current_location in all_murmur_locations

False

In [9]:
current_patient_data

'50635 4 4000\nAV 50635_AV.hea 50635_AV.wav 50635_AV.tsv\nPV 50635_PV.hea 50635_PV.wav 50635_PV.tsv\nTV 50635_TV.hea 50635_TV.wav 50635_TV.tsv\nMV 50635_MV.hea 50635_MV.wav 50635_MV.tsv\n#Age: nan\n#Sex: Female\n#Height: nan\n#Weight: nan\n#Pregnancy status: True\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Normal\n#Campaign: CC2014\n#Additional ID: nan\n'

In [8]:
murmur = get_murmur(current_patient_data)
murmur

'Absent'

In [ ]:
    current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)

In [1]:
from helper_code import *
from team_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import os
import tqdm
import numpy as np
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
# Find data files.

data_folder = "./data/dataset/data/training_data/"

In [2]:
patient_files = find_patient_files(data_folder)

In [3]:
patient_files

['./data/dataset/data/training_data/2530.txt',
 './data/dataset/data/training_data/9979.txt',
 './data/dataset/data/training_data/9983.txt',
 './data/dataset/data/training_data/13918.txt',
 './data/dataset/data/training_data/14241.txt',
 './data/dataset/data/training_data/14998.txt',
 './data/dataset/data/training_data/23625.txt',
 './data/dataset/data/training_data/24160.txt',
 './data/dataset/data/training_data/29045.txt',
 './data/dataset/data/training_data/29378.txt',
 './data/dataset/data/training_data/31737.txt',
 './data/dataset/data/training_data/33151.txt',
 './data/dataset/data/training_data/36327.txt',
 './data/dataset/data/training_data/38337.txt',
 './data/dataset/data/training_data/39043.txt',
 './data/dataset/data/training_data/39403.txt',
 './data/dataset/data/training_data/39456.txt',
 './data/dataset/data/training_data/40058.txt',
 './data/dataset/data/training_data/40798.txt',
 './data/dataset/data/training_data/40840.txt',
 './data/dataset/data/training_data/43852.t

In [7]:
pat_dat = load_patient_data(patient_files[0])

In [8]:
get_murmur(pat_dat)

'Absent'

In [9]:
get_outcome(pat_dat)

'Abnormal'

In [16]:
cv_murmur = []
cv_outcome = []
cv_filename = []
for i in tqdm.tqdm(range(len(patient_files))):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    cv_murmur.append(get_murmur(current_patient_data))
    cv_outcome.append(get_outcome(current_patient_data))
cv_outcome = np.asarray(cv_outcome)
cv_murmur = np.asarray(cv_murmur)
current_patient_data = np.asarray(current_patient_data)

100%|██████████| 942/942 [00:00<00:00, 4529.19it/s]


In [21]:
current_patient_data

'85349 3 4000\nAV 85349_AV.hea 85349_AV.wav 85349_AV.tsv\nPV 85349_PV.hea 85349_PV.wav 85349_PV.tsv\nTV 85349_TV.hea 85349_TV.wav 85349_TV.tsv\n#Age: nan\n#Sex: Female\n#Height: nan\n#Weight: nan\n#Pregnancy status: True\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Normal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [17]:
skf = StratifiedKFold(n_splits=5)

In [18]:
skf.split(cv_murmur, cv_outcome)

<generator object _BaseKFold.split at 0x0000013564308CF0>

In [20]:
for train_index, val_index in skf.split(cv_murmur, cv_outcome):
    print(np.unique(cv_murmur[train_index],return_counts=True))
    print(np.unique(cv_outcome[train_index], return_counts=True))

(array(['Absent', 'Present', 'Unknown'], dtype='<U7'), array([559, 137,  57], dtype=int64))
(array(['Abnormal', 'Normal'], dtype='<U8'), array([364, 389], dtype=int64))
(array(['Absent', 'Present', 'Unknown'], dtype='<U7'), array([547, 150,  56], dtype=int64))
(array(['Abnormal', 'Normal'], dtype='<U8'), array([365, 388], dtype=int64))
(array(['Absent', 'Present', 'Unknown'], dtype='<U7'), array([554, 144,  56], dtype=int64))
(array(['Abnormal', 'Normal'], dtype='<U8'), array([365, 389], dtype=int64))
(array(['Absent', 'Present', 'Unknown'], dtype='<U7'), array([554, 150,  50], dtype=int64))
(array(['Abnormal', 'Normal'], dtype='<U8'), array([365, 389], dtype=int64))
(array(['Absent', 'Present', 'Unknown'], dtype='<U7'), array([566, 135,  53], dtype=int64))
(array(['Abnormal', 'Normal'], dtype='<U8'), array([365, 389], dtype=int64))


In [ ]:
def get_data(patient_files, data_folder, new_frequenzy, num_murmur_classes, num_outcome_classes):
    data = []
    murmurs = list()
    outcomes = list()
    for i in tqdm.tqdm(range(len(patient_files))):
        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        current_recordings, freq = load_recordings(data_folder, current_patient_data, get_frequencies=True)
        for j in range(len(current_recordings)):
            data.append(signal.resample(current_recordings[j], int((len(current_recordings[j])/freq[j]) * new_frequenzy)))
            current_auscultation_location = current_patient_data.split('\n')[1:len(current_recordings)+1][j].split(" ")[0]
            all_murmur_locations = get_murmur_locations(current_patient_data).split("+")
            current_murmur = np.zeros(num_murmur_classes, dtype=int)
            if get_murmur(current_patient_data) == "Present":
                if current_auscultation_location in all_murmur_locations:
                    current_murmur[0] = 1
                else:
                    pass
            elif get_murmur(current_patient_data) == "Unknown":
                current_murmur[1] = 1
            elif get_murmur(current_patient_data) == "Absent":
                current_murmur[2] = 1
            murmurs.append(current_murmur)

            current_outcome = np.zeros(num_outcome_classes, dtype=int)
            outcome = get_outcome(current_patient_data)
            if outcome in outcome_classes:
                j = outcome_classes.index(outcome)
                current_outcome[j] = 1
            outcomes.append(current_outcome)
    data_padded = pad_array(data, (32256//2))
    data_padded = np.expand_dims(data_padded,2)

    murmurs = np.vstack(murmurs)
    outcomes = np.argmax(np.vstack(outcomes),axis=1)

    return data_padded, murmurs, outcomes

In [22]:
patient_files

['./data/dataset/data/training_data/2530.txt',
 './data/dataset/data/training_data/9979.txt',
 './data/dataset/data/training_data/9983.txt',
 './data/dataset/data/training_data/13918.txt',
 './data/dataset/data/training_data/14241.txt',
 './data/dataset/data/training_data/14998.txt',
 './data/dataset/data/training_data/23625.txt',
 './data/dataset/data/training_data/24160.txt',
 './data/dataset/data/training_data/29045.txt',
 './data/dataset/data/training_data/29378.txt',
 './data/dataset/data/training_data/31737.txt',
 './data/dataset/data/training_data/33151.txt',
 './data/dataset/data/training_data/36327.txt',
 './data/dataset/data/training_data/38337.txt',
 './data/dataset/data/training_data/39043.txt',
 './data/dataset/data/training_data/39403.txt',
 './data/dataset/data/training_data/39456.txt',
 './data/dataset/data/training_data/40058.txt',
 './data/dataset/data/training_data/40798.txt',
 './data/dataset/data/training_data/40840.txt',
 './data/dataset/data/training_data/43852.t

In [23]:
NEW_FREQUENCY = 250
max_len = 0
for i in tqdm.tqdm(range(len(patient_files))):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings, freq = load_recordings(data_folder, current_patient_data, get_frequencies=True)
    for rec in current_recordings:
        rec_len = (len(rec)/freq) * NEW_FREQUENCY
        if rec_len > max_len:
            max_len = rec_len

  0%|          | 0/942 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for /: 'int' and 'list'

In [24]:
rec_len

NameError: name 'rec_len' is not defined

In [28]:
freq

[4000, 4000, 4000, 4000]

In [27]:
len(rec)/freq

TypeError: unsupported operand type(s) for /: 'int' and 'list'

In [35]:
for indx, rec in enumerate(current_recordings):
    print(len(rec)//freq[indx]*NEW_FREQUENCY)

5750
3250
3500
6500
